# Dataset

Ako prvotné testovacie dáta sme použili voľne dostupný dataset zo stránky 
https://github.com/JingchunCheng/All-Age-Faces-Dataset?fbclid=IwAR3sesRO_P9-hGeQN2xfupIsEbIuUNNH2Ul_wPLgaH32d-O6ivTsBIv567A

Dataset obsahuje 13 322 fotiek z toho 7 381 je žien a 5 941 mužov. Vekové rozpätie je od 2 do 80 rokov. 
Dataset poskytuje aj upravené/zarovnané/otočené fotky, aby bolo lepšie vidieť do tváre. Pri testovaní, 
či je lepšie používať originály fotky alebo upravená verzia vyšli lepšie výsledky pre originály 
v porovnaní dosiahnutej presnosti.

Dáta sú už predom rozdelené na testovacie a trénovacie presne na polovicu.Predtým ako sa fotky pošlu do siete sa 
musia trochu upraviť. Fotky sú upravované funkciami opencv napr. resize() a delené rgb kanály hodnotou 255, 
aby hodnoty boli v rozsahu 0-1. Každá fotka má iné rozlíšenie, tzn. že musíme ich pretransformovať 
na rovnaký formát pomocou funkcie resize(). Názov fotky už obsahuje potrebné informácie ako rok aj pohlavie. 
Je vo formáte XXXXXAYY.jpg, kde XXXXX je poradové číslo a ked je menšie ako 7 382 tak sa jedná o ženu, 
a keď je väčšie rovné tak je to muž. YY je vek človeka na fotke. Vytvorili sme vekové kategórie 
(0-4),(5-9),(10-17),(18-25),(26-39),(40-59),(60-74),(75-), ktoré reprezentujú čísla od 0-7 (8 kategórii).

Keďže je v datasete len 13 322 fotiek a sú rozdelené na polovicu, neurónová sieť sa veľmi pomaly učila aj pri 10 epochách. 
Z toho dôvodu sme k týmto dátam pribalili druhý dataset z https://susanqq.github.io/UTKFace/?fbclid=IwAR2IuscY8oJ6ubuUdKCMRgCtZ0ZQJ38VMMykgbwK7WZK3LLvszBGp26HOyY 
Dataset poskytuje 23 708 fotiek ľudí od veku 1 do 116. V názve fotografie už máme pripravené pohlavie a vek človeka na fotke. 
Tento dataset sme rozdelili na 3/4 trénovaciu jednotku a 1/4 na testovaciu. Výsledné polia fotiek sme spojili spolu s predošlými fotkami. 
Výsledna trénovacia jednotka má veľkosť 24 418 fotiek. Pri trénovaní s 30timi epochami sme dosiahli nasledovné výsledky.

24418/24418 - 196s 8ms/step - loss: 0.0808 - accuracy: 0.9764 - val_loss: 5.7291 - val_accuracy: 0.4167
Model: "neural_network"
_________________________________________________________________
Layer (type)                 Output Shape              Param  

=================================================================
conv2d_1 (Conv2D)            (None, 75, 85, 16)        2368      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 37, 42, 16)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 37, 42, 32)        12832     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 18, 21, 32)        0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 18, 21, 64)        18496     
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 9, 10, 64)         0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 5760)              0         
_________________________________________________________________
dense_1 (Dense)              (None, 512)               2949632   
_________________________________________________________________
dense_2 (Dense)              (None, 16)                8208      

=================================================================
Total params: 2,991,536
Trainable params: 2,991,536
Non-trainable params: 0
_________________________________________________________________


In [ ]:
import glob
from os.path import basename
import numpy as np
import random as ra
from cv2 import *

HEIGHT = 85
WIDTH = 75
DIM = (HEIGHT, WIDTH)


def prep_img(image):
    image = resize(image, DIM)
    image = image / 255.0
    image = np.array(image)
    return image


def decide_age(age):
    if age < 5:
        return 0
    if age < 10:
        return 1
    if age < 18:
        return 2
    if age < 26:
        return 3
    if age < 40:
        return 4
    if age < 60:
        return 5
    if age < 75:
        return 6
    return 7


def prep_label(age, number):
    number = int(number)
    age = int(age)
    if number < 7381:
        return decide_age(age=age)
    return decide_age(age=age) + 8


def get_age_gender(name):
    name = basename(name)
    if name[1] == '_':
        age = int(name[0])
        gender = int(name[2])
    elif name[2] == '_':
        age = int(name[0] + name[1])
        gender = int(name[3])
    else:
        age = int(name[0] + name[1] + name[2])
        gender = int(name[4])
    if gender == 1:
        age = decide_age(age=age)
    else:
        age = decide_age(age=age) + 8
    return age


def get_photos(file, images, labels):
    f = open(file, "r")

    for x in f:
        img = imread("D:\\Nastavenia\\Dokumenty\\STU\\4rocnik\\neu\\ns\\python\\photos\\original_images\\" + x[0:12])
        images.append(prep_img(image=img))
        labels.append(prep_label(age=x[6:8], number=x[0:5]))
    f.close()
    return


def get_data():
    print("loading first dataset...")
    train_images1, train_labels1, test_images1, test_labels1 = get_data1()
    print("loading second dataset...")
    train_images2, train_labels2, test_images2, test_labels2 = get_data2()

    print("preparing dataset...")
    train_images = np.concatenate((train_images1, train_images2))
    train_labels = np.concatenate((train_labels1, train_labels2))
    test_images = np.concatenate((test_images1, test_images2))
    test_labels = np.concatenate((test_labels1, test_labels2))

    train_images = np.array(train_images)
    train_labels = np.array(train_labels)
    test_images = np.array(test_images)
    test_labels = np.array(test_labels)

    return train_images, train_labels, test_images, test_labels


def get_data1():
    train_labels = []
    train_images = []
    test_images = []
    test_labels = []

    get_photos(file="D:\\Nastavenia\\Dokumenty\\STU\\4rocnik\\neu\\ns\\python\\photos\\image_sets\\train.txt",
               images=train_images, labels=train_labels)
    get_photos(file="D:\\Nastavenia\\Dokumenty\\STU\\4rocnik\\neu\\ns\\python\\photos\\image_sets\\val.txt",
               images=test_images, labels=test_labels)

    train_images = np.array(train_images)
    train_labels = np.array(train_labels)
    test_images = np.array(test_images)
    test_labels = np.array(test_labels)

    return train_images, train_labels, test_images, test_labels


def get_data2():
    train_labels = []
    train_images = []
    test_images = []
    test_labels = []

    # DIMENSION (200,200,3)
    images = [prep_img(imread(file)) for file in glob.glob("..\\photos\\mixed\\UTKFace\\*.jpg")]
    labels = [get_age_gender(file) for file in glob.glob("..\\photos\\mixed\\UTKFace\\*.jpg")]

    pom = list(zip(images, labels))
    ra.shuffle(pom)
    images, labels = zip(*pom)

    for x in range(0, int(len(images) * 3 / 4)):
        train_images.append(images[x])
        train_labels.append(labels[x])
    for x in range(int(len(images) * 3 / 4), len(images)):
        test_images.append(images[x])
        test_labels.append(labels[x])

    train_images = np.array(train_images)
    train_labels = np.array(train_labels)
    test_images = np.array(test_images)
    test_labels = np.array(test_labels)

    return train_images, train_labels, test_images, test_labels


# Model

Model pozostáva z 3 konvolučných vrstiev, 3 MaxPooling vrstiev a z 2 full-mesh (Dense) vrstiev.
Ako aktivačnú funkciu používame relu pretože sa pri testovaní ukázala ako rýchlejšia a lepšia.

<img src="model.png">

In [ ]:
from keras.models import Sequential
from keras.layers import Conv2D, Dense, MaxPooling2D, Flatten


class MyNeuralNetwork(Sequential):
    def __init__(self, height, width):
        super(MyNeuralNetwork, self).__init__(name='MyNeuralNetwork')
        self.add(
            Conv2D(
                input_shape=(width, height, 3),
                filters=16,
                kernel_size=7,
                padding='same',
                activation='relu')
        )
        self.add(
            MaxPooling2D(pool_size=(2, 2))
        )
        self.add(
            Conv2D(
                filters=32,
                kernel_size=5,
                padding='same',
                activation='relu')
        )
        self.add(
            MaxPooling2D(pool_size=(2, 2))
        )
        self.add(
            Conv2D(
                filters=64,
                kernel_size=3,
                padding='same',
                activation='relu')
        )
        self.add(
            MaxPooling2D(pool_size=(2, 2))
        )
        self.add(
            Flatten()
        )
        self.add(
            Dense(
                units=512,
                activation='relu')
        )
        self.add(
            Dense(
                units=16,
                activation='softmax')
        )

# Trénovanie

In [ ]:
from src.model import MyNeuralNetwork
from keras.optimizers import Adam
from src.import_data import *

BATCH_SIZE = 16
EPOCHS = 30

train_images, train_labels, test_images, test_labels = get_data()

model = MyNeuralNetwork(HEIGHT, WIDTH)

model.compile(
    optimizer=Adam(learning_rate=0.001),
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy'])

callbacks = [
    # TensorBoard(
    #     log_dir="logs/" + timestamp(),
    #     histogram_freq=1,
    #     profile_batch=0)
]

model.fit(
    x=train_images,
    y=train_labels,
    batch_size=BATCH_SIZE,
    shuffle=True,
    validation_data=(test_images, test_labels),
    callbacks=callbacks,
    epochs=EPOCHS)

model.save("model.h5")
model.summary()